In [3]:
from config import api_key
from yelpapi import YelpAPI
import requests
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt

In [4]:
cities = ["Anaheim,CA", "Santa Ana,CA", "Irvine,CA", "Huntington Beach,CA", "Garden Grove,CA", "Orange,CA", "Fullerton,CA", "Costa Mesa,CA", "Mission Viejo,CA","Westminster,CA"]
url = "https://api.yelp.com/v3/businesses/search"
headers = {'Authorization': 'Bearer %s' %api_key}
price_locations = {}

for city in cities:
    locations={}
    params = {"term":"restaurant",
              "location":city,
              "limit":50,
             "radius":5000}
    responses=requests.get(url,headers = headers, params = params).json()
    for response in responses["businesses"]:
        try:
            money= response["price"]
            if money not in locations:
                locations[money]=1
                
            else:
                locations[money]+=1
        except KeyError:
            continue  
    price_locations[city]=locations 
    url_params = {"term":"restaurant",
                  "location":city,
                  "limit":50,
                 "offset":50,
                  "radius":5000}
    results = requests.get(url,headers = headers, params = url_params).json()
    for r in results["businesses"]:
        try:
            m = r["price"]
            if m not in locations:
                locations[m]=1
            else:
                locations[m]+=1
        except KeyError:
            continue
    price_locations[city]=locations
print(price_locations)        
     

{'Anaheim,CA': {'$$': 63, '$$$': 8, '$': 26, '$$$$': 3}, 'Santa Ana,CA': {'$$': 48, '$$$$': 1, '$': 49}, 'Irvine,CA': {'$$': 64, '$$$': 2, '$': 33}, 'Cypress,CA': {'$$': 47, '$': 50, '$$$': 2}, 'Garden Grove,CA': {'$$': 49, '$': 48, '$$$$': 1}, 'Orange,CA': {'$': 44, '$$': 52, '$$$$': 2, '$$$': 1}, 'Fullerton,CA': {'$$': 60, '$': 35, '$$$': 2}, 'Costa Mesa,CA': {'$$': 67, '$': 26, '$$$': 4, '$$$$': 2}, 'Tustin,CA': {'$': 37, '$$': 60, '$$$': 3}, 'Westminster,CA': {'$': 59, '$$': 40}}


In [6]:
columns = ["Cheap","Affordable","Expensive","Luxury"]
DF=pd.DataFrame.from_dict(price_locations).T #.T means transpose, switch columns and rows
DF = DF.reset_index()
# df.rename(columns = {'index':'City','$':'Cheap'}, inplace = True)
df = DF.rename(columns={"$":"Cheap","$$":"Affordable","$$$":"Expensive","$$$$":"Luxury","index":"City"})
# DF.to_csv("output.csv")

df = df.fillna(0)
df

,City,Cheap,Affordable,Expensive,Luxury
0,"Anaheim,CA",26.0,63.0,8.0,3.0
1,"Costa Mesa,CA",26.0,67.0,4.0,2.0
2,"Cypress,CA",50.0,47.0,2.0,0.0
3,"Fullerton,CA",35.0,60.0,2.0,0.0
4,"Garden Grove,CA",48.0,49.0,0.0,1.0
5,"Irvine,CA",33.0,64.0,2.0,0.0
6,"Orange,CA",44.0,52.0,1.0,2.0
7,"Santa Ana,CA",49.0,48.0,0.0,1.0
8,"Tustin,CA",37.0,60.0,3.0,0.0
9,"Westminster,CA",59.0,40.0,0.0,0.0
